# Resnet18 with pretrain

## Pre Process

In [7]:
import pandas as pd
import cv2
import numpy as np

In [8]:
data=pd.read_csv('../data_animals10/class_path.csv',index_col=0)

channel_sampling=[[],[],[]]

for i in range(data.shape[0]):
    rand_idx=np.random.random()
    if rand_idx>=0.95:
        _, path_i = data.iloc[i, :]
        img_i = cv2.imread(path_i, 1)/255
        channel_sampling[0]+=img_i[:,:,0].reshape(-1).tolist()
        channel_sampling[1]+=img_i[:,:,1].reshape(-1).tolist()
        channel_sampling[2]+=img_i[:,:,2].reshape(-1).tolist()

print('channel 1\'s mean= {:.3f} , std = {:.3f}'.format(np.mean(channel_sampling[0]),np.std(channel_sampling[0])))
print('channel 2\'s mean= {:.3f} , std = {:.3f}'.format(np.mean(channel_sampling[1]),np.std(channel_sampling[1])))
print('channel 3\'s mean= {:.3f} , std = {:.3f}'.format(np.mean(channel_sampling[2]),np.std(channel_sampling[2])))

channel 1's mean= 0.413 , std = 0.280
channel 2's mean= 0.504 , std = 0.268
channel 3's mean= 0.527 , std = 0.279


## Pretrain step

In [1]:
from baseline_resnet18 import ResNet18
from Pretrain_resnet18 import clf_head,Rotation,Color

#Use the ResNet18 on Cifar-10
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from Pretrain_resnet18 import Animals10
import torchvision.transforms as transforms
import cv2
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter


from sklearn.metrics import accuracy_score,f1_score

### rotation

In [2]:
# check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set hyperparameter
EPOCH = 25
pre_epoch = 0
BATCH_SIZE = 50
LR = 0.00005

# prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128, 128]),
    transforms.RandomCrop(128, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.422, 0.495, 0.515), (0.299, 0.285, 0.289)) # 抽样算出来的
])

transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128, 128]),
    transforms.ToTensor(),
    transforms.Normalize((0.422, 0.495, 0.515), (0.299, 0.285, 0.289))
])

map_idx = {'butterfly': 0,
           'cat': 1,
           'chicken': 2,
           'class_path.csv': 3,
           'cow': 4,
           'dog': 5,
           'elephant': 6,
           'horse': 7,
           'sheep': 8,
           'spider': 9,
           'squirrel': 10}

trainset_rotation = Animals10('../data_animals10/class_path.csv', train=True,
                              transform=transform_train, target_map=map_idx, rotation=True, train_val=[0.6, 0.4])
trainloader_rotation = torch.utils.data.DataLoader(
    trainset_rotation, batch_size=BATCH_SIZE, shuffle=False)

testset_rotation = Animals10('../data_animals10/class_path.csv', train=False,
                             transform=transform_train, target_map=map_idx, rotation=True, train_val=[0.6, 0.4])
testloader_rotation = torch.utils.data.DataLoader(
    testset_rotation, batch_size=100, shuffle=False)

# define ResNet18
baseline_model = ResNet18().to(device)
rotation = Rotation(512, 9, 4).to(device)

# define loss funtion & optimizer
loss_f_rotation = nn.CrossEntropyLoss()
optimizer = optim.Adam([{'params': baseline_model.parameters()}, {
                       'params': rotation.parameters()}], lr=LR, weight_decay=5e-4)

writer = SummaryWriter('../logs/Rotation_ResNet18')

In [3]:
# train
for epoch in range(pre_epoch, EPOCH):
    print('epoch:', epoch)
    print('Train start')

    baseline_model.train()
    rotation.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    length = len(trainloader_rotation)
    for i, data in enumerate(trainloader_rotation):

        # prepare dataset
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.long().to(device)
        optimizer.zero_grad()

        # forward & backward
        outputs = baseline_model(inputs)
        outputs = rotation(outputs)

        loss = loss_f_rotation(outputs, labels)
        loss.backward()
        optimizer.step()

        # print ac & loss in each batch
        with torch.no_grad():
            outputs_target = outputs.detach().cpu().numpy().argmax(axis=1)
            labels_check = labels.detach().cpu().numpy()
            acc_train = accuracy_score(labels_check, outputs_target)
            f1_train = f1_score(labels_check, outputs_target, average='macro')

            writer.add_scalar("Train/Loss",
                              loss.item(), (i + 1 + epoch * length))
            writer.add_scalar("Train/Acc",
                              acc_train, (i + 1 + epoch * length))
            writer.add_scalar("Train/F1_score(macro)",
                              f1_train, (i + 1 + epoch * length))

    print('Train finish')

    print('Test start')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader_rotation:
            baseline_model.eval()
            rotation.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = baseline_model(images)
            outputs = rotation(outputs)
            predicted = outputs.detach().argmax(axis=1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        writer.add_scalar("Test/Acc",
                          correct/total, epoch)
    print('Test finish')
    if (epoch+1) % 5 == 0:
        torch.save(
            baseline_model, '../model_path/pretrain_resnet18_rotation_{}.pt'.format(epoch))

print('All train finish, Acc in test is ', (correct/total).item())

epoch: 0
Train start
Train finish
Test start
Test finish
epoch: 1
Train start
Train finish
Test start
Test finish
epoch: 2
Train start
Train finish
Test start
Test finish
epoch: 3
Train start
Train finish
Test start
Test finish
epoch: 4
Train start
Train finish
Test start
Test finish
epoch: 5
Train start
Train finish
Test start
Test finish
epoch: 6
Train start
Train finish
Test start
Test finish
epoch: 7
Train start
Train finish
Test start
Test finish
epoch: 8
Train start
Train finish
Test start
Test finish
epoch: 9
Train start
Train finish
Test start
Test finish
epoch: 10
Train start
Train finish
Test start
Test finish
epoch: 11
Train start
Train finish
Test start
Test finish
epoch: 12
Train start
Train finish
Test start
Test finish
epoch: 13
Train start
Train finish
Test start
Test finish
epoch: 14
Train start
Train finish
Test start
Test finish
epoch: 15
Train start
Train finish
Test start
Test finish
epoch: 16
Train start
Train finish
Test start
Test finish
epoch: 17
Train start
Tr

### Color

In [5]:
# check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set hyperparameter
EPOCH = 25
pre_epoch = 0
BATCH_SIZE = 50
LR = 0.00005

# prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128, 128]),
    transforms.RandomCrop(128, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.422, 0.495, 0.515), (0.299, 0.285, 0.289))
])

transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128, 128]),
    transforms.ToTensor(),
    transforms.Normalize((0.422, 0.495, 0.515), (0.299, 0.285, 0.289))
])

map_idx = {'butterfly': 0,
           'cat': 1,
           'chicken': 2,
           'class_path.csv': 3,
           'cow': 4,
           'dog': 5,
           'elephant': 6,
           'horse': 7,
           'sheep': 8,
           'spider': 9,
           'squirrel': 10}

trainset_color = Animals10('../data_animals10/class_path.csv', train=True,
                           transform=transform_train, target_map=map_idx, color=True, train_val=[0.6, 0.4])
trainloader_color = torch.utils.data.DataLoader(
    trainset_color, batch_size=BATCH_SIZE, shuffle=False)

testset_color = Animals10('../data_animals10/class_path.csv', train=False,
                             transform=transform_train, target_map=map_idx, color=True, train_val=[0.6, 0.4])
testloader_color = torch.utils.data.DataLoader(
    testset_color, batch_size=100, shuffle=False)

# define ResNet18
baseline_model = ResNet18().to(device)
color = Color(512, 2).to(device)

# define loss funtion & optimizer
loss_f_color = nn.MSELoss()
optimizer = optim.Adam([{'params': baseline_model.parameters()}, {
                       'params': color.parameters()}], lr=LR, weight_decay=5e-4)

writer = SummaryWriter('../logs/Color_ResNet18')


In [6]:
# train
for epoch in range(pre_epoch, EPOCH):
    print('epoch:', epoch)
    print('Train start')
    baseline_model.train()
    color.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    length = len(trainloader_color)
    for i, data in enumerate(trainloader_color):

        # prepare dataset
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward & backward
        outputs = baseline_model(inputs)
        outputs = color(outputs)

        loss = loss_f_color(outputs, labels)
        loss.backward()
        optimizer.step()

        # print ac & loss in each batch
        with torch.no_grad():
            writer.add_scalar("Train/Loss",
                              loss.item(), (i + 1 + epoch * length))

    print('Train finish')

    print('Test start')
    with torch.no_grad():
        mse = 0
        total = 0
        for data in testloader_color:
            baseline_model.eval()
            color.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = baseline_model(images)
            outputs = color(outputs)
            mse += loss_f_color(outputs, labels)*labels.size(0)
            total += labels.size(0)
        writer.add_scalar("Test/Loss",
                          mse/total, epoch)
    print('Test finish')
    if (epoch+1) % 5 == 0:
        torch.save(baseline_model,
                   '../model_path/pretrain_resnet18_color_{}.pt'.format(epoch))


print('All train finish, Loss in test is ', (mse/total).item())

epoch: 0
Train start
Train finish
Test start
Test finish
epoch: 1
Train start
Train finish
Test start
Test finish
epoch: 2
Train start
Train finish
Test start
Test finish
epoch: 3
Train start
Train finish
Test start
Test finish
epoch: 4
Train start
Train finish
Test start
Test finish
epoch: 5
Train start
Train finish
Test start
Test finish
epoch: 6
Train start
Train finish
Test start
Test finish
epoch: 7
Train start
Train finish
Test start
Test finish
epoch: 8
Train start
Train finish
Test start
Test finish
epoch: 9
Train start
Train finish
Test start
Test finish
epoch: 10
Train start
Train finish
Test start
Test finish
epoch: 11
Train start
Train finish
Test start
Test finish
epoch: 12
Train start
Train finish
Test start
Test finish
epoch: 13
Train start
Train finish
Test start
Test finish
epoch: 14
Train start
Train finish
Test start
Test finish
epoch: 15
Train start
Train finish
Test start
Test finish
epoch: 16
Train start
Train finish
Test start
Test finish
epoch: 17
Train start
Tr

## Linear Classification Protocol step

### rotation

In [2]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set hyperparameter
EPOCH = 25
pre_epoch = 0
BATCH_SIZE = 50
LR = 0.00005

#prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.RandomCrop(128, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='../data_CIFAR10', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data_CIFAR10', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#labels in CIFAR10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

baseline_model=torch.load('../model_path/pretrain_resnet18_rotation_24.pt')

clf=clf_head(512,9,10).to(device)

#define loss funtion & optimizer
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(clf.parameters(), lr=LR, weight_decay=5e-4)

writer = SummaryWriter('../logs/ResNet18_LCP_rotation')


Files already downloaded and verified
Files already downloaded and verified


In [3]:
# train
for epoch in range(pre_epoch, EPOCH):
    print('epoch:', epoch)
    print('Train start')

    clf.train()

    for i, data in enumerate(trainloader, 0):
        # prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward & backward
        outputs = baseline_model(inputs)
        outputs = clf(outputs)
        loss = loss_f(outputs, labels)
        loss.backward()
        optimizer.step()

        # print ac & loss in each batch
        with torch.no_grad():
            outputs_target = outputs.detach().cpu().numpy().argmax(axis=1)
            labels_check = labels.detach().cpu().numpy()
            acc_train = accuracy_score(labels_check, outputs_target)
            f1_train = f1_score(labels_check, outputs_target, average='macro')

            writer.add_scalar("Train/Loss",
                              loss.item(), (i + 1 + epoch * length))
            writer.add_scalar("Train/Acc",
                              acc_train, (i + 1 + epoch * length))
            writer.add_scalar("Train/F1_score(macro)",
                              f1_train, (i + 1 + epoch * length))

    # get the ac with testdataset in each epoch
    print('Train finish')

    print('Test start')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
 
            clf.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = baseline_model(images)
            outputs = clf(outputs)
            predicted = outputs.detach().argmax(axis=1)

            total += labels.size(0)
            correct += (predicted == labels).sum()

        writer.add_scalar("Test/Acc",
                          correct/total, epoch)
    print('Test finish')

print('All train finish, Acc in test is ', (correct/total).item())

torch.save(clf,'../model_path/resnet18_rotation_clf_head.pt')


epoch: 0
Train start
Train finish
Test start
Test finish
epoch: 1
Train start
Train finish
Test start
Test finish
epoch: 2
Train start
Train finish
Test start
Test finish
epoch: 3
Train start
Train finish
Test start
Test finish
epoch: 4
Train start
Train finish
Test start
Test finish
epoch: 5
Train start
Train finish
Test start
Test finish
epoch: 6
Train start
Train finish
Test start
Test finish
epoch: 7
Train start
Train finish
Test start
Test finish
epoch: 8
Train start
Train finish
Test start
Test finish
epoch: 9
Train start
Train finish
Test start
Test finish
epoch: 10
Train start
Train finish
Test start
Test finish
epoch: 11
Train start
Train finish
Test start
Test finish
epoch: 12
Train start
Train finish
Test start
Test finish
epoch: 13
Train start
Train finish
Test start
Test finish
epoch: 14
Train start
Train finish
Test start
Test finish
epoch: 15
Train start
Train finish
Test start
Test finish
epoch: 16
Train start
Train finish
Test start
Test finish
epoch: 17
Train start
Tr

### Color

In [4]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set hyperparameter
EPOCH = 25
pre_epoch = 0
BATCH_SIZE = 50
LR = 0.00005

#prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.RandomCrop(128, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='../data_CIFAR10', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data_CIFAR10', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#labels in CIFAR10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

baseline_model=torch.load('../model_path/pretrain_resnet18_color_24.pt')

clf=clf_head(512,9,10).to(device)

#define loss funtion & optimizer
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(clf.parameters(), lr=LR, weight_decay=5e-4)

writer = SummaryWriter('../logs/ResNet18_LCP_color')


Files already downloaded and verified
Files already downloaded and verified


In [5]:
# train
for epoch in range(pre_epoch, EPOCH):
    print('epoch:', epoch)
    print('Train start')

    clf.train()

    for i, data in enumerate(trainloader, 0):
        # prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward & backward
        outputs = baseline_model(inputs)
        outputs = clf(outputs)
        loss = loss_f(outputs, labels)
        loss.backward()
        optimizer.step()

        # print ac & loss in each batch
        with torch.no_grad():
            outputs_target = outputs.detach().cpu().numpy().argmax(axis=1)
            labels_check = labels.detach().cpu().numpy()
            acc_train = accuracy_score(labels_check, outputs_target)
            f1_train = f1_score(labels_check, outputs_target, average='macro')

            writer.add_scalar("Train/Loss",
                              loss.item(), (i + 1 + epoch * length))
            writer.add_scalar("Train/Acc",
                              acc_train, (i + 1 + epoch * length))
            writer.add_scalar("Train/F1_score(macro)",
                              f1_train, (i + 1 + epoch * length))

    # get the ac with testdataset in each epoch
    print('Train finish')

    print('Test start')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:

            clf.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = baseline_model(images)
            outputs = clf(outputs)
            predicted = outputs.detach().argmax(axis=1)

            total += labels.size(0)
            correct += (predicted == labels).sum()

        writer.add_scalar("Test/Acc",
                          correct/total, epoch)
    print('Test finish')

print('All train finish, Acc in test is ', (correct/total).item())

torch.save(clf,'../model_path/resnet18_color_clf_head.pt')


epoch: 0
Train start
Train finish
Test start
Test finish
epoch: 1
Train start
Train finish
Test start
Test finish
epoch: 2
Train start
Train finish
Test start
Test finish
epoch: 3
Train start
Train finish
Test start
Test finish
epoch: 4
Train start
Train finish
Test start
Test finish
epoch: 5
Train start
Train finish
Test start
Test finish
epoch: 6
Train start
Train finish
Test start
Test finish
epoch: 7
Train start
Train finish
Test start
Test finish
epoch: 8
Train start
Train finish
Test start
Test finish
epoch: 9
Train start
Train finish
Test start
Test finish
epoch: 10
Train start
Train finish
Test start
Test finish
epoch: 11
Train start
Train finish
Test start
Test finish
epoch: 12
Train start
Train finish
Test start
Test finish
epoch: 13
Train start
Train finish
Test start
Test finish
epoch: 14
Train start
Train finish
Test start
Test finish
epoch: 15
Train start
Train finish
Test start
Test finish
epoch: 16
Train start
Train finish
Test start
Test finish
epoch: 17
Train start
Tr